In [62]:
import pandas as pd
import os
import requests
import sys
from io import StringIO
from datetime import datetime, timedelta


'''
This script is designed to create a sleep log file for the group analysis of accelerometer data.
It reads the individual participant files by first matching labID with studyID, then builds path to individual files on the RDSS
Aggregates the sleep data by participant and session (with _accel suffix) and saves it to a CSV file.
'''

token = 'DE4E2DB72778DACA9B8848574107D2F5'
INT_DIR = '/mnt/lss/Projects/BOOST/InterventionStudy/3-experiment/data/act-int-test'
OBS_DIR = '/mnt/lss/Projects/BOOST/ObservationalStudy/3-experiment/data/act-obs-test'
rdss_dir = '/mnt/rdss/VossLab/Repositories/Accelerometer_Data/WearTime'

def compare_ids(rdss_dir, token, daysago=350):
    """
    Pulls all files from RDSS
    Pulls the list from RedCap
    Compares IDs and returns a dictionary with two keys:
        - 'matches': normal matches mapping boost_id to a list of dicts (filename, labID, date)
        - 'duplicates': a list of dictionaries each with lab_id, boost_id, filenames (list), and dates (list)
    """
    # Retrieve the RedCap report and duplicates from report
    report, report_duplicates = _return_report(token)
    # Retrieve the full RDSS file list and duplicate files merged with duplicates from report
    rdss, file_duplicates = _rdss_file_list(report_duplicates)

    # Initialize the result dictionary for normal (non-duplicate) matches
    result = {}

    # Iterate over the rows in the cleaned RedCap report
    for _, row in report.iterrows():
        boost_id = str(row['boost_id'])
        lab_id = str(row['lab_id'])
        
        # Find matching files in the RDSS list
        rdss_matches = rdss[rdss['ID'] == lab_id]
        if not rdss_matches.empty:
            if boost_id not in result:
                result[boost_id] = []
            for _, match_row in rdss_matches.iterrows():
                result[boost_id].append({
                    'filename': match_row['filename'],
                    'labID': lab_id,
                    'date': match_row['Date']
                })
    
    # Process duplicates into the desired structure.
    duplicates_dict = []
    if not file_duplicates.empty:
        # Group by lab_id and boost_id; each group represents one duplicate combination.
        grouped = file_duplicates.groupby(['lab_id', 'boost_id'])
        for (lab_id, boost_id), group in grouped:
            duplicates_dict.append({
                'lab_id': lab_id,
                'boost_id': boost_id,
                'filenames': group['filename'].tolist(),
                'dates': group['Date'].tolist()
            })
    else:
        print("Found no duplicates.")

    return {'matches': result, 'duplicates': duplicates_dict}

def _return_report(token):
    """
    pulls the id report from the rdss via redcap api.
    reads the report as a dataframe.
    checks for boost_ids that are associated with multiple lab_ids, logs a critical error,
    and removes these rows from the dataframe.
    separates duplicate rows (based on any column) from the cleaned data.
    
    returns:
        df_cleaned: dataframe with duplicates removed and problematic boost_ids excluded
        duplicate_rows: dataframe of duplicate rows
    """
    url = 'https://redcap.icts.uiowa.edu/redcap/api/'
    data = {
        'token': token,
        'content': 'report',
        'report_id': 43327,
        'format': 'csv'
    }
    r = requests.post(url, data=data)
    if r.status_code != 200:
        print(f"error! status code is {r.status_code}")
        sys.exit(1)
    
    df = pd.read_csv(StringIO(r.text))
    
    # identify boost_ids associated with multiple lab_ids.
    boost_id_counts = df.groupby('boost_id')['lab_id'].nunique()
    problematic_boost_ids = boost_id_counts[boost_id_counts > 1].index.tolist()
    
    if problematic_boost_ids:
        print(f"found boost_id(s) with multiple lab_ids: {', '.join(map(str, problematic_boost_ids))}. "
                        "these entries will be removed from processing.")
        df = df[~df['boost_id'].isin(problematic_boost_ids)]
    
    # identify and separate duplicate rows based on any column.
    duplicate_rows = df[df.duplicated(keep=False)]
    df_cleaned = df.drop_duplicates(keep=False)
    
    if not duplicate_rows.empty:
        print(f"duplicate rows found:\n{duplicate_rows}")
    
    return df_cleaned, duplicate_rows

def _rdss_file_list(duplicates, daysago=None):
    """
    extracts the first string before the space and the date from filenames ending with .csv
    in the specified folder and stores them in a dataframe.
    
    Also, merges the file list with duplicate report entries based on lab_id.
    
    Returns:
        df: DataFrame of all file entries
        merged_df: DataFrame of file entries that match duplicate lab_ids from the report
    """
    extracted_data = []

    # Loop through all files in the rdss_dir folder.
    for filename in os.listdir(rdss_dir):
        if filename.endswith('.csv'):
            try:
                # Handle both old and new filename formats
                if '_' in filename and filename.endswith('.csv'):
                    # New format: 1288_4-26-2025_Sleep.csv
                    parts = filename.replace('.csv', '').split('_')
                    if len(parts) >= 3:
                        base_name = parts[0]  # lab_id
                        date_part = parts[1]  # date
                        extracted_data.append({'ID': base_name, 'Date': date_part, 'filename': filename})
                    else:
                        print(f"Skipping file with unexpected format: {filename}")
                else:
                    try:
                        base_name = filename.split(' ')[0]  # Extract lab_id (old format)
                        date_part = filename.split('(')[1].split(')')[0]  # Extract date (old format)
                        extracted_data.append({'ID': base_name, 'Date': date_part, 'filename': filename})
                    except IndexError:
                        print(f"Skipping file with unexpected format: {filename}")
            except IndexError:
                print(f"Skipping file with unexpected format: {filename}")

    df = pd.DataFrame(extracted_data)

    if not df.empty:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

        if daysago:
            cutoff_date = datetime.today() - timedelta(days=daysago)
            df = df[df['Date'] >= cutoff_date]  # Filter files within the last `daysago` days
        else:
            df = df[df['Date'] >= '2024-08-05']  # Filter out rows before the threshold date

    # Filter the file list to only include rows where ID is in the duplicate report (if any)
    if not duplicates.empty:
        matched_df = df[df['ID'].isin(duplicates['lab_id'])]
        # Merge with the duplicates to bring in boost_id information from the report
        merged_df = matched_df.merge(duplicates, left_on='ID', right_on='lab_id')
    else:
        merged_df = pd.DataFrame()

    return df, merged_df

matches = compare_ids(rdss_dir, token, daysago=None)
# Print the matches and duplicates for verification
print("Matches:")
for boost_id, files in matches['matches'].items():
    print(f"Boost ID: {boost_id}")
    for file_info in files:
        print(f"  - {file_info['filename']} (Lab ID: {file_info['labID']}, Date: {file_info['date']})")
print("\nDuplicates:")
for dup in matches['duplicates']:
    print(f"Lab ID: {dup['lab_id']}, Boost ID: {dup['boost_id']}")
    print(f"  Filenames: {', '.join(dup['filenames'])}")
    print(f"  Dates: {', '.join(map(str, dup['dates']))}")
# The above code is a complete script that compares IDs from RDSS and RedCap, identifies matches and duplicates, and prints the results.

'''
Below we create the sessions, where if the same subject ID has multiple files, we will create a session for each file ordered by date.
this will now be stored as a dataframe with the columns:
    # 'subject_id', 'session_id', 'filename', 'date'
    # where subject_id is 'sub-<subject_id>', session_id is 'ses-<session_number>', filename is the file name with full path, and date is the date of the file.
'''
def create_sessions(matches):
    """
    Create sessions from the matches dictionary.
    
    Args:
        matches (dict): Dictionary containing matches with boost_id as keys and list of file info as values.
    
    Returns:
        pd.DataFrame: DataFrame with columns 'subject_id', 'session_id', 'filename', 'date'.
    """
    sessions = []
    
    for boost_id, files in matches['matches'].items():
        subject_id = f'sub-{boost_id}'
        for i, file_info in enumerate(files):
            session_id = f'ses-{i + 1}'  # Session number starts from 1
            sessions.append({
                'subject_id': subject_id,
                'session_id': session_id,
                'filename': os.path.join(rdss_dir, file_info['filename']),
                'date': file_info['date']
            })
    
    return pd.DataFrame(sessions)

# Create sessions from the Matches
sessions_df = create_sessions(matches)
# Print the sessions DataFrame for verification
print("\nSessions DataFrame:")
print(sessions_df)


found boost_id(s) with multiple lab_ids: 7023. these entries will be removed from processing.
Skipping file with unexpected format: 498_WearTime.csv
Skipping file with unexpected format: 633_WearTime.csv
Skipping file with unexpected format: 619_WearTime.csv
Skipping file with unexpected format: 92_WearTime.csv
Skipping file with unexpected format: 555_WearTime.csv
Skipping file with unexpected format: 649_WearTime.csv
Skipping file with unexpected format: 557_WearTime.csv
Skipping file with unexpected format: 504_WearTime.csv
Skipping file with unexpected format: 207_WearTime.csv
Skipping file with unexpected format: AP_WearTime.csv
Skipping file with unexpected format: 648_WearTime.csv
Skipping file with unexpected format: 589_WearTime.csv
Skipping file with unexpected format: 505_WearTime.csv
Skipping file with unexpected format: 579_WearTime.csv
Skipping file with unexpected format: 634_WearTime.csv
Skipping file with unexpected format: 500_WearTime.csv
Skipping file with unexpecte

/tmp/nix-shell.RrgKp6/ipykernel_184880/761771891.py:154: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


In [77]:
sessions_df.to_clipboard()

In [64]:
s8022_test = [pd.read_csv(sessions_df.iloc[0]['filename'],skiprows=1), pd.read_csv(sessions_df.iloc[1]['filename'],skiprows=1)]

In [65]:
test_s1 = s8022_test[0]
test_s2 = s8022_test[1]
test_s1

,Date,Wear Time (minutes),Non-Wear Time (minutes),Wear %,Non-Wear %,Vector Magnitude
0,2/7/2025,1321,119,91.7,8.3,1060314.3
1,2/8/2025,1439,1,99.9,0.1,1997886.7
2,2/9/2025,1440,0,100,0,1882234.5
3,2/10/2025,1440,0,100,0,1647210.0
4,2/11/2025,1440,0,100,0,1820595.0
5,2/12/2025,1440,0,100,0,1382595.7
6,2/13/2025,1440,0,100,0,2210102.8
7,2/14/2025,1440,0,100,0,1688519.2
8,2/15/2025,1348,92,93.6,6.4,1319762.8
9,2/16/2025,0,1440,0,100,1586.9


In [78]:
# 2) Find the index of the row where your second header lives
hdr_idx = test_s1.index[test_s1.iloc[:, 0] == "Date/Time Start"][0]

# 3) Pull out that row’s values to use as your new column names
new_header = test_s1.iloc[hdr_idx]

# 4) Drop everything up through that header row, reset the index
test_s1 = test_s1.iloc[hdr_idx+1 : ].reset_index(drop=True)

# 5) Assign the new_header as the DataFrame’s columns
test_s1.columns = new_header

IndexError: index 0 is out of bounds for axis 0 with size 0

In [67]:
test_s1

10,Date/Time Start,Date/Time Stop,Wear or Non-Wear,Length (minutes),Use?,NaN
0,2/7/2025 12:00 AM,2/7/2025 6:55 AM,Wear,415,TRUE,NaN
1,2/7/2025 6:55 AM,2/7/2025 7:28 AM,Non-Wear,33,FALSE,NaN
2,2/7/2025 7:28 AM,2/7/2025 5:21 PM,Wear,593,TRUE,NaN
3,2/7/2025 5:21 PM,2/7/2025 6:47 PM,Non-Wear,86,FALSE,NaN
4,2/7/2025 6:47 PM,2/8/2025 10:05 AM,Wear,918,TRUE,NaN
5,2/8/2025 10:05 AM,2/8/2025 10:06 AM,Non-Wear,1,FALSE,NaN
6,2/8/2025 10:06 AM,2/15/2025 10:28 PM,Wear,10822,TRUE,NaN
7,2/15/2025 10:28 PM,2/17/2025 12:00 AM,Non-Wear,1532,FALSE,NaN


In [68]:
test_s1 = test_s1[['Date/Time Start', 'Date/Time Stop', 'Wear or Non-Wear']]

In [69]:
test_s1 = test_s1[test_s1['Wear or Non-Wear']=='Non-Wear']

In [70]:
nonwear_df = test_s1
nonwear_df

10,Date/Time Start,Date/Time Stop,Wear or Non-Wear
1,2/7/2025 6:55 AM,2/7/2025 7:28 AM,Non-Wear
3,2/7/2025 5:21 PM,2/7/2025 6:47 PM,Non-Wear
5,2/8/2025 10:05 AM,2/8/2025 10:06 AM,Non-Wear
7,2/15/2025 10:28 PM,2/17/2025 12:00 AM,Non-Wear


In [76]:
int_sleep = pd.read_csv('/mnt/lss/Projects/BOOST/InterventionStudy/3-experiment/data/act-int-test/sleep_log_intervention.csv')
int_sleep.to_clipboard()

In [72]:
sleep_test = int_sleep[int_sleep['ID']=='sub-8022_ses-1_accel']
sleep_test.to_clipboard()
sleep_df = sleep_test

In [73]:
import pandas as pd

# 1) parse your nonwear datetimes and extract just the date
nonwear_df['Date/Time Start'] = pd.to_datetime(nonwear_df['Date/Time Start'])
nonwear_df['Date/Time Stop' ] = pd.to_datetime(nonwear_df['Date/Time Stop'])
nonwear_df['nw_date']       = nonwear_df['Date/Time Start'].dt.date

# 2) build a dict: date → list of (start_time_str, stop_time_str)
nw_dict = {
    d: list(zip(
        grp['Date/Time Start'].dt.strftime('%H:%M:%S'),
        grp['Date/Time Stop' ].dt.strftime('%H:%M:%S'),
    ))
    for d, grp in nonwear_df.groupby('nw_date')
}

# 3) make sure your sleep dates are actual date objects
for i in range(1, 11):
    col = f'D{i}_date'
    if col in sleep_df.columns:
        sleep_df[col] = pd.to_datetime(sleep_df[col]).dt.date

# 4) for each row and each day, pluck out the intervals and write new cols
for idx, row in sleep_df.iterrows():
    for i in range(1, 11):
        date_col = f'D{i}_date'
        if pd.isna(row.get(date_col)):
            continue

        day = row[date_col]
        intervals = nw_dict.get(day, [])

        for j, (off_time, on_time) in enumerate(intervals, start=1):
            off_col = f'D{i}_nonwear{j}_off'
            on_col  = f'D{i}_nonwear{j}_on'

            # create the column if it doesn’t exist yet
            if off_col not in sleep_df.columns:
                sleep_df[off_col] = pd.NA
            if on_col not in sleep_df.columns:
                sleep_df[on_col]  = pd.NA

            # assign the times
            sleep_df.at[idx, off_col] = off_time
            sleep_df.at[idx, on_col ] = on_time



/tmp/nix-shell.RrgKp6/ipykernel_184880/3756629236.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_df[col] = pd.to_datetime(sleep_df[col]).dt.date
/tmp/nix-shell.RrgKp6/ipykernel_184880/3756629236.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_df[col] = pd.to_datetime(sleep_df[col]).dt.date
/tmp/nix-shell.RrgKp6/ipykernel_184880/3756629236.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [75]:
sleep_df

,ID,D1_date,D1_inbed,D1_wakeup,D2_date,D2_inbed,D2_wakeup,D3_date,D3_inbed,D3_wakeup,...,D10_inbed,D10_wakeup,D1_nonwear1_off,D1_nonwear1_on,D1_nonwear2_off,D1_nonwear2_on,D2_nonwear1_off,D2_nonwear1_on,D8_nonwear1_off,D8_nonwear1_on
0,sub-8022_ses-1_accel,2025-02-07,19:00:00,08:30:00,2025-02-08,23:45:00,07:30:00,2025-02-09,23:55:00,07:00:00,...,NaN,NaN,06:55:00,07:28:00,17:21:00,18:47:00,10:05:00,10:06:00,22:28:00,00:00:00


In [80]:
import pandas as pd

def integrate_nonwear(sleep_df: pd.DataFrame,
                      wear_files_df: pd.DataFrame,
                      id_col: str = "ID") -> pd.DataFrame:
    """
    Updates sleep_df in‑place by reading each wear‑time CSV in wear_files_df,
    extracting Non‑Wear intervals, and appending them to the matching row
    in sleep_df under D{day}_nonwear{n}_{off,on} columns.
    
    wear_files_df must have columns: subject_id, session_id, filename, date
    sleep_df must have an ID column like "sub-8022_ses-1_accel" and
      D1_date … D10_date columns of type date or datetime.
    """
    for _, wf in wear_files_df.iterrows():
        # build the sleep_df ID key
        key = f"{wf.subject_id}_{wf.session_id}_accel"

        # 1) read raw CSV and splice out the non-wear table
        raw = pd.read_csv(wf.filename, header=0, dtype=str)
        # locate the second-header row
        mask = (raw.iloc[:,0]
                .astype(str)
                .str.strip()
                .str.contains("Date/Time Start", na=False))
        if not mask.any():
            continue  # no header found, skip
        hdr_idx = mask.idxmax()
        new_hdr = raw.iloc[hdr_idx]
        df = raw.iloc[hdr_idx+1 : ].reset_index(drop=True)
        df.columns = new_hdr
        df = df[['Date/Time Start','Date/Time Stop','Wear or Non-Wear']]
        nonwear = df[df['Wear or Non-Wear']=='Non-Wear'].copy()

        # 2) parse datetimes and group by date
        nonwear['Date/Time Start'] = pd.to_datetime(nonwear['Date/Time Start'])
        nonwear['Date/Time Stop']  = pd.to_datetime(nonwear['Date/Time Stop'])
        nonwear['nw_date']         = nonwear['Date/Time Start'].dt.date
        nw_dict = (
            nonwear
            .groupby('nw_date')
            .apply(lambda g: list(zip(
                g['Date/Time Start'].dt.strftime('%H:%M:%S'),
                g['Date/Time Stop'].dt.strftime('%H:%M:%S'),
            )))
            .to_dict()
        )

        # 3) find matching sleep_df row
        matches = sleep_df.index[sleep_df[id_col] == key]
        if len(matches) == 0:
            continue
        ridx = matches[0]

        # 4) for each D{i}_date, assign its intervals
        for i in range(1, 11):
            date_col = f'D{i}_date'
            if date_col not in sleep_df.columns:
                break
            raw_date = sleep_df.at[ridx, date_col]
            if pd.isna(raw_date):
                continue
            day = pd.to_datetime(raw_date).date()
            intervals = nw_dict.get(day, [])
            for j, (off_t, on_t) in enumerate(intervals, start=1):
                off_col = f'D{i}_nonwear{j}_off'
                on_col  = f'D{i}_nonwear{j}_on'
                # create columns if missing
                if off_col not in sleep_df:
                    sleep_df[off_col] = pd.NA
                if on_col not in sleep_df:
                    sleep_df[on_col]  = pd.NA
                sleep_df.at[ridx, off_col] = off_t
                sleep_df.at[ridx, on_col ] = on_t

    return sleep_df


In [81]:
# sleep_df: your DataFrame of all sleep logs
# wear_files_df: DataFrame with columns [subject_id, session_id, filename, date]
int_sleep = pd.read_csv('/mnt/lss/Projects/BOOST/InterventionStudy/3-experiment/data/act-int-test/sleep_log_intervention.csv')

updated = integrate_nonwear(int_sleep, sessions_df)


In [82]:
updated

,ID,D1_date,D1_inbed,D1_wakeup,D2_date,D2_inbed,D2_wakeup,D3_date,D3_inbed,D3_wakeup,...,D7_wakeup,D8_date,D8_inbed,D8_wakeup,D9_date,D9_inbed,D9_wakeup,D10_date,D10_inbed,D10_wakeup
0,sub-8022_ses-1_accel,2025-02-07,19:00:00,08:30:00,2025-02-08,23:45:00,07:30:00,2025-02-09,23:55:00,07:00:00,...,07:00:00,2025-02-15,01:00:00,09:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-8022_ses-2_accel,2025-04-17,00:00:00,06:30:00,2025-04-18,23:10:00,07:40:00,2025-04-20,02:45:00,07:30:00,...,08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-8001_ses-1_accel,2024-09-21,00:30:00,08:30:00,2024-09-22,00:15:00,07:40:00,2024-09-23,05:20:00,09:30:00,...,09:00:00,2024-09-28,00:30:00,08:45:00,2024-09-29,00:30:00,08:15:00,2024-09-30,00:20:00,08:30:00
3,sub-8002_ses-1_accel,2024-09-14,22:15:00,06:45:00,2024-09-15,23:00:00,06:45:00,2024-09-16,22:51:00,06:45:00,...,07:00:00,2024-09-21,22:30:00,06:45:00,2024-09-22,22:45:00,06:45:00,NaN,NaN,NaN
4,sub-8002_ses-2_accel,2024-11-07,23:00:00,03:30:00,2024-11-08,21:30:00,05:30:00,2024-11-09,21:30:00,05:30:00,...,06:15:00,2024-11-14,22:45:00,06:15:00,2024-11-15,22:30:00,07:00:00,NaN,NaN,NaN
5,sub-8002_ses-3_accel,2025-02-26,23:00:00,06:30:00,2025-02-27,23:00:00,06:30:00,2025-02-28,23:00:00,07:30:00,...,07:00:00,2025-03-05,22:30:00,06:30:00,2025-03-06,23:00:00,06:30:00,NaN,NaN,NaN
6,sub-8002_ses-4_accel,2025-05-05,22:30:00,06:30:00,2025-05-06,22:30:00,06:30:00,2025-05-07,23:00:00,07:00:00,...,06:45:00,2025-05-12,23:15:00,07:00:00,2025-05-13,22:30:00,06:45:00,NaN,NaN,NaN
7,sub-8003_ses-1_accel,2024-10-05,22:30:00,07:30:00,2024-10-06,21:30:00,06:00:00,2024-10-07,22:00:00,06:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,sub-8003_ses-2_accel,2025-02-01,22:50:00,07:00:00,2025-02-02,22:00:00,06:15:00,2025-02-03,22:30:00,05:30:00,...,07:15:00,2025-02-08,23:00:00,07:00:00,2025-02-09,22:30:00,06:15:00,NaN,NaN,NaN
9,sub-8004_ses-1_accel,2024-09-26,18:45:00,05:55:00,2024-09-27,22:00:00,07:01:00,2024-09-28,21:57:00,06:12:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
updated.to_csv('./sleep_w_nonwear.csv')